##### Importation des librairies

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor,GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA

from sklearn.metrics import (
    f1_score, precision_score, recall_score, roc_auc_score,
    confusion_matrix, balanced_accuracy_score, precision_recall_curve,
    auc, make_scorer
)
from imblearn.metrics import geometric_mean_score


from sklearn.preprocessing import StandardScaler, OneHotEncoder,  FunctionTransformer # MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from datetime import datetime, timedelta
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectFromModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.compose import ColumnTransformer
from config import Config, Constant
import os, sys
import shutil
import warnings
warnings.filterwarnings('ignore')

In [2]:
resultat_compile=dict()

In [3]:
df=pd.read_csv(os.path.join(Config.DATASET_DIR,Config.DATA_1DEK))
df_cleaned = df.dropna(subset=['Label Secheresse'])
df.columns

Index(['Station', 'Year', 'Month', 'Decade', 'v_wind_925', 'u_wind_850',
       'u_wind_700', 'u_wind_200', 'eau_precipitable', 't_point_rosee',
       'h_vol_sol_wat', 'anom_lef_dek', 'anom_nino_dek', 'Date',
       'Label Secheresse', 'Saison_Pluie'],
      dtype='object')

In [4]:
#features=['v_wind_925','u_wind_850','u_wind_700','u_wind_200','eau_precipitable','t_point_rosee','h_vol_sol_wat','anom_lef_dek','anom_nino_dek']
#feature_rnn_dek=['v_wind_925','u_wind_850','u_wind_700','u_wind_200','eau_precipitable','t_point_rosee','h_vol_sol_wat','anom_lef_dek','anom_nino_dek','Label Secheresse']
#feature_rnn_mon=['v_wind_925','u_wind_850','u_wind_700','u_wind_200','eau_precipitable','t_point_rosee','h_vol_sol_wat','anom_lef_mois','anom_nino_mois','Label Secheresse']

feature_dek=['v_wind_925','u_wind_850','u_wind_700','u_wind_200','eau_precipitable','t_point_rosee','h_vol_sol_wat','anom_lef_dek','anom_nino_dek']

In [5]:
df_cleaned.head()

,Station,Year,Month,Decade,v_wind_925,u_wind_850,u_wind_700,u_wind_200,eau_precipitable,t_point_rosee,h_vol_sol_wat,anom_lef_dek,anom_nino_dek,Date,Label Secheresse,Saison_Pluie
0,Bobo_Dioulasso,1961.0,1.0,1.0,-5.685185,-4.901044,-6.507310,24.359285,5.872143e-09,274.505384,0.177588,0.365944,-0.024139,1961-01-01,0.0,False
1,Bobo_Dioulasso,1961.0,1.0,2.0,-5.627637,-1.549461,-1.188504,28.219953,1.950160e-08,274.479259,0.178358,0.232241,0.064786,1961-01-02,0.0,False
2,Bobo_Dioulasso,1961.0,1.0,3.0,-7.595885,-0.394695,2.207007,32.303318,-7.126785e-07,273.833101,0.178193,0.668866,0.241058,1961-01-03,0.0,False
3,Bobo_Dioulasso,1961.0,2.0,1.0,-5.607470,-3.607335,3.229098,12.524779,-3.006926e-08,269.842157,0.172408,0.866864,0.408497,1961-02-01,0.0,False
4,Bobo_Dioulasso,1961.0,2.0,2.0,-5.117176,-3.535849,3.110321,19.788904,-4.638627e-08,269.390330,0.173361,1.098664,0.390542,1961-02-02,0.0,False


In [6]:
# Créer un répertoire de sortie s'il n'existe pas déjà
dir_resultat = Config.RESULT_DIR
# Supprimer le dossier existant, y compris son contenu
if os.path.exists(dir_resultat):
    shutil.rmtree(dir_resultat)
# Créer un nouveau répertoire de sortie
os.makedirs(dir_resultat, exist_ok=True)

In [7]:
# Créer un répertoire de sortie s'il n'existe pas déjà
dir_resultat_sel_var = Config.RESULT_DIR_SEL_VAR
# Supprimer le dossier existant, y compris son contenu
if os.path.exists(dir_resultat_sel_var):
    shutil.rmtree(dir_resultat_sel_var)
# Créer un nouveau répertoire de sortie
os.makedirs(dir_resultat_sel_var, exist_ok=True)

In [8]:
df.head()

,Station,Year,Month,Decade,v_wind_925,u_wind_850,u_wind_700,u_wind_200,eau_precipitable,t_point_rosee,h_vol_sol_wat,anom_lef_dek,anom_nino_dek,Date,Label Secheresse,Saison_Pluie
0,Bobo_Dioulasso,1961.0,1.0,1.0,-5.685185,-4.901044,-6.507310,24.359285,5.872143e-09,274.505384,0.177588,0.365944,-0.024139,1961-01-01,0.0,False
1,Bobo_Dioulasso,1961.0,1.0,2.0,-5.627637,-1.549461,-1.188504,28.219953,1.950160e-08,274.479259,0.178358,0.232241,0.064786,1961-01-02,0.0,False
2,Bobo_Dioulasso,1961.0,1.0,3.0,-7.595885,-0.394695,2.207007,32.303318,-7.126785e-07,273.833101,0.178193,0.668866,0.241058,1961-01-03,0.0,False
3,Bobo_Dioulasso,1961.0,2.0,1.0,-5.607470,-3.607335,3.229098,12.524779,-3.006926e-08,269.842157,0.172408,0.866864,0.408497,1961-02-01,0.0,False
4,Bobo_Dioulasso,1961.0,2.0,2.0,-5.117176,-3.535849,3.110321,19.788904,-4.638627e-08,269.390330,0.173361,1.098664,0.390542,1961-02-02,0.0,False


#### Creation des fonctions utiles

In [9]:
# Définition de la fonction to_numeric_with_nan
def to_numeric_with_nan(value):
    try:
        return int(value)
    except (ValueError, TypeError):
        return pd.NA  # Retourne une valeur manquante

In [10]:
def label_change(value):
    if pd.isna(value):
        return pd.NA
    elif value == 0.0:
        return 0
    elif value == 1.0:
        return 1
    else:
        return value  # Si la valeur est différente de NaN, 0.0 et 1.0, la renvoyer telle quelle

In [11]:
# Fonction pour supprimer les colonnes spécifiées
def drop_columns(X, columns_to_drop):
    return X.drop(columns_to_drop, axis=1)

# Colonnes à supprimer
columns_to_drop = ['Station', 'Year', 'Month', 'Decade', 'Saison_Pluie','Date']
#drop_columns_transformer = FunctionTransformer(drop_columns, kw_args={'columns_to_drop': columns_to_drop})

#### Creation de pipeline pour la gestion des variables numeriques et categorielles

In [12]:
# Fonction pour supprimer les colonnes spécifiées
def drop_columns(X, columns_to_drop):
    return X.drop(columns_to_drop, axis=1)

# Colonnes à supprimer
columns_to_drop = ['Station', 'Year', 'Month', 'Decade', 'Saison_Pluie', 'Date']

In [13]:
# Définir les features numériques (remplacez 'feature_dek' par votre liste réelle de colonnes numériques)
numeric_features = feature_dek

# Pipeline pour les features numériques
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Stratified K-Fold cross-validation configuration
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize SMOTE
smote = SMOTE(random_state=42)

# Combine transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ])

# Pipeline principal
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('feature_selector', SelectFromModel(RandomForestClassifier(random_state=42), threshold='median')),
    ('classifier', RandomForestClassifier(random_state=42))
])


In [14]:
# Paramètres à tester pour GridSearch (ajoutez des hyperparamètres pour PCA si nécessaire)
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4]
}

# Scorer basé sur le F1-Score
scorer = make_scorer(f1_score, pos_label=1)

# Initialisation de GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring=scorer,
    cv=StratifiedKFold(n_splits=4, shuffle=True, random_state=42),
    n_jobs=-1,
    verbose=2
)


In [15]:
#def rnf_train_evaluate(df, numeric_features, target_column, label_change, columns_to_drop, smote, skf, grid_search):
def rnf_train_evaluate(df, numeric_features,spei, localite):
    data = df.copy()

    # Séparation des caractéristiques (X) et de la cible (y)
    X = data.drop('Label Secheresse', axis=1)  # Remplacez 'Label Secheresse' par le nom de votre colonne cible
    y = data['Label Secheresse'].apply(label_change)  # Assurez-vous que label_change est défini

    # Drop columns before applying SMOTE
    X = drop_columns(X, columns_to_drop)    

    # Dictionnaire pour stocker les métriques pour chaque pli
    metrics = {
        'f1': [],
        'precision': [],
        'recall': [],
        'roc_auc': [],
        'balanced_acc': [],
        'g_mean': [],
        'pr_auc': []
    }

    # Liste pour stocker les trois principales caractéristiques pour chaque pli
    top_features_list = []

    # Validation croisée avec SMOTE et GridSearchCV
    for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Application de SMOTE sur les données d'entraînement uniquement
        X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
        
        # Ajustement de GridSearchCV sur les données d'entraînement avec SMOTE
        grid_search.fit(X_train_smote, y_train_smote)
        
        # Prédiction sur les données de test avec le meilleur estimateur
        y_pred = grid_search.best_estimator_.predict(X_test)
        y_pred_proba = grid_search.best_estimator_.predict_proba(X_test)[:, 1]
        
        # Calcul des différentes métriques
        metrics['f1'].append(f1_score(y_test, y_pred, pos_label=1))
        metrics['precision'].append(precision_score(y_test, y_pred, pos_label=1))
        metrics['recall'].append(recall_score(y_test, y_pred, pos_label=1))
        metrics['roc_auc'].append(roc_auc_score(y_test, y_pred_proba))
        metrics['balanced_acc'].append(balanced_accuracy_score(y_test, y_pred))
        metrics['g_mean'].append(geometric_mean_score(y_test, y_pred))
        
        precision_vals, recall_vals, _ = precision_recall_curve(y_test, y_pred_proba)
        metrics['pr_auc'].append(auc(recall_vals, precision_vals))
        
        # Extraction de l'importance des caractéristiques et sélection des 3 principales
        feature_selector = grid_search.best_estimator_.named_steps['feature_selector']
        model = feature_selector.estimator_
        
        importances = model.feature_importances_
        important_indices = np.argsort(importances)[::-1][:3]
        feature_names = X.columns
        top_features = feature_names[important_indices].tolist()
        
        top_features_list.append(top_features)

    # Calcul des moyennes et des écarts types des métriques
    avg_metrics = {metric: f"{localite}: {np.mean(values):.4f} ± {np.std(values):.4f}" for metric, values in metrics.items()}
    resultat_compile[f'Random forest_{localite}_{spei}']=avg_metrics

    return avg_metrics, top_features_list


In [16]:
grouped_data = df_cleaned.groupby('Station')
for station, group_data in grouped_data:
    rnf_train_evaluate(group_data, feature_dek,'1decade', station)

Fitting 4 folds for each of 108 candidates, totalling 432 fits
Fitting 4 folds for each of 108 candidates, totalling 432 fits


Fitting 4 folds for each of 108 candidates, totalling 432 fits
Fitting 4 folds for each of 108 candidates, totalling 432 fits


Fitting 4 folds for each of 108 candidates, totalling 432 fits
Fitting 4 folds for each of 108 candidates, totalling 432 fits


Fitting 4 folds for each of 108 candidates, totalling 432 fits
Fitting 4 folds for each of 108 candidates, totalling 432 fits


Fitting 4 folds for each of 108 candidates, totalling 432 fits
Fitting 4 folds for each of 108 candidates, totalling 432 fits


...

Fitting 4 folds for each of 108 candidates, totalling 432 fits
Fitting 4 folds for each of 108 candidates, totalling 432 fits


Fitting 4 folds for each of 108 candidates, totalling 432 fits
Fitting 4 folds for each of 108 candidates, totalling 432 fits


Fitting 4 folds for each of 108 candidates, totalling 432 fits
Fitting 4 folds for each of 108 candidates, totalling 432 fits


Fitting 4 folds for each of 108 candidates, totalling 432 fits
Fitting 4 folds for each of 108 candidates, totalling 432 fits


Fitting 4 folds for each of 108 candidates, totalling 432 fits
Fitting 4 folds for each of 108 candidates, totalling 432 fits


In [ ]:
grouped_data = data_ext_1dek.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_model_ranf(group_data, '1decade', 'extr', station)

### A garder pour l'instant

In [ ]:

# Fonction pour supprimer les colonnes spécifiées
def drop_columns(X, columns_to_drop):
    return X.drop(columns_to_drop, axis=1)

# Colonnes à supprimer
columns_to_drop = ['Station', 'Year', 'Month', 'Decade', 'Saison_Pluie', 'Date']

# Définir les features numériques (remplacez 'feature_dek' par votre liste réelle de colonnes numériques)
numeric_features = feature_dek

# Pipeline pour les features numériques
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Stratified K-Fold cross-validation configuration
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize SMOTE
smote = SMOTE(random_state=42)

# Combine transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ])

# Pipeline principal
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('feature_selector', SelectFromModel(RandomForestClassifier(random_state=42), threshold='median')),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Paramètres à tester pour GridSearch (ajoutez des hyperparamètres pour PCA si nécessaire)
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4]
}

# Scorer basé sur le F1-Score
scorer = make_scorer(f1_score, pos_label=1)

# Initialisation de GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring=scorer,
    cv=StratifiedKFold(n_splits=4, shuffle=True, random_state=42),
    n_jobs=-1,
    verbose=2
)

# Example data loading
data = df_cleaned.copy()  # Remplacez avec vos données réelles
data = data[data['Station'] == 'Gaoua']

X = data.drop('Label Secheresse', axis=1)  # Remplacez 'Label Secheresse' par le nom de votre colonne cible
y = data['Label Secheresse'].apply(label_change)  # Assurez-vous que label_change est défini

# Drop columns before applying SMOTE
X = drop_columns(X, columns_to_drop)

# Store metrics for each fold
metrics = {
    'f1': [],
    'precision': [],
    'recall': [],
    'roc_auc': [],
    'balanced_acc': [],
    'g_mean': [],
    'pr_auc': []
}

# List to store the top 3 features for each fold
top_features_list = []

# Cross-validation with SMOTE and GridSearchCV
for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Apply SMOTE on the training data only
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    
    # Fit GridSearchCV on the SMOTE-applied training data
    grid_search.fit(X_train_smote, y_train_smote)
    
    # Predict on the test data using the best estimator
    y_pred = grid_search.best_estimator_.predict(X_test)
    y_pred_proba = grid_search.best_estimator_.predict_proba(X_test)[:, 1]
    
    # Calculate the different metrics
    metrics['f1'].append(f1_score(y_test, y_pred, pos_label=1))
    metrics['precision'].append(precision_score(y_test, y_pred, pos_label=1))
    metrics['recall'].append(recall_score(y_test, y_pred, pos_label=1))
    metrics['roc_auc'].append(roc_auc_score(y_test, y_pred_proba))
    metrics['balanced_acc'].append(balanced_accuracy_score(y_test, y_pred))
    metrics['g_mean'].append(geometric_mean_score(y_test, y_pred))
    
    precision_vals, recall_vals, _ = precision_recall_curve(y_test, y_pred_proba)
    metrics['pr_auc'].append(auc(recall_vals, precision_vals))
    
    # Extract the feature importance and top 3 features
    feature_selector = grid_search.best_estimator_.named_steps['feature_selector']
    model = feature_selector.estimator_
    
    importances = model.feature_importances_
    important_indices = np.argsort(importances)[::-1][:3]
    feature_names = X.columns
    top_features = feature_names[important_indices].tolist()
    
    top_features_list.append(top_features)

# Displaying the average metrics across folds
for metric, values in metrics.items():
    print(f"Average {metric}: {np.mean(values):.4f} ± {np.std(values):.4f}")

# Display the top 3 features from each fold
print("Top 3 features in each fold:")
for fold, top_features in enumerate(top_features_list, 1):
    print(f"Fold {fold}: {top_features}")


### Imbalance data management

In [ ]:
def create_rnn_model(X_train, y_train, X_valid, y_valid, X_test, y_test, spei, type_sec, localite):
    input_shape = X_train[0].shape  # La forme d'entrée est déterminée par les données d'entraînement
    model = keras.Sequential([
        keras.layers.LSTM(64, input_shape=input_shape),
        keras.layers.Dense(1, activation='sigmoid')  # Couche de sortie avec une seule unité et activation sigmoid pour la classification binaire
    ])

    # Compilez le modèle avec la perte et l'optimiseur spécifiés
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_valid, y_valid))

    # Évaluez le modèle sur les données de test
    y_pred_proba = model.predict(X_test)
    y_pred = (y_pred_proba > 0.5).astype(int)  # Convertir les probabilités en classes binaires (0 ou 1)

    # Calculez les métriques
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    # Calculez le Mean Squared Error (MSE) et le Mean Absolute Error (MAE)
    #mse = tf.keras.losses.mean_squared_error(y_test, y_pred_proba).numpy()
    #mae = tf.keras.losses.mean_absolute_error(y_test, y_pred_proba).numpy()


    resultat_compile[f'RNN_{localite}_{spei}_{type_sec}'] = {
        'Mean Accuracy': round(np.mean([accuracy]), 3),
        'Precision': round(precision, 3),
        'Recall': round(recall, 3),
        'F1 Score': round(f1, 3),
        'ROC AUC': round(roc_auc, 3),
        #'loss': round(loss, 3),
        #'mse': round(np.mean([mse]), 3),
        #'mae': round(np.mean([mae]), 3)
    }
    columns = ['Algo', 'Spei', 'Type_Sec', 'Station', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC AUC']

    # Création du DataFrame avec des colonnes vides
    df = pd.DataFrame(columns=columns)

    metrics_obtenu = ['RNN', spei, type_sec, localite, np.mean([accuracy]), round(precision, 3), round(recall, 3),
                      round(f1, 3), round(roc_auc, 3)]
    # Ajout d'une nouvelle ligne au DataFrame
    df.loc[len(df)] = metrics_obtenu
    result_file_csv = os.path.join(dir_resultat, f'rnn_{spei}_{type_sec}_{localite}.csv')
    result_file_excel = os.path.join(dir_resultat, f'rnn_{spei}_{type_sec}_{localite}.xlsx')

    df.to_csv(result_file_csv, index=False)
    df.to_excel(result_file_excel, index=False)

    return model

In [ ]:
def prepare_data_for_rnn(df_data, feature_columns, time_columns, target_column, sequence_length, test_size=0.2, valid_size=0.5):
    dataframe=df_data.copy()
    # Étape 1 : Triez par la colonne temporelle
    #dataframe.sort_values(by=time_columns [ 'Station',time_columns], ascending=[True, True], inplace=True)
    if 'Decade' in dataframe.columns:
        dataframe.sort_values(by=[ "Year", "Month", "Decade"], ascending=[True, True, True], inplace=True)
    else:
        dataframe.sort_values(by=["Year", "Month"], ascending=[True, True], inplace=True)



    # Étape 2 : Sélection des caractéristiques et de la cible
    features = feature_columns
    target = target_column
    dataframe[target]=dataframe[target].astype(int)

    # Étape 3 : Création de séquences temporelles
    X, y = [], []

    for i in range(len(dataframe) - sequence_length):
        X.append(dataframe[features].values[i:i+sequence_length])
        y.append(dataframe[target].values[i+sequence_length])

    X = np.array(X)
    y = np.array(y)

    # Aplatir les données
    X = X.reshape(X.shape[0], -1)  # Aplatissement en 2D

    # Étape 4 : Normalisation
    scaler = StandardScaler()  #MinMaxScaler()
    X = scaler.fit_transform(X)

    # Étape 5 : Division des données
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=test_size, shuffle=False)
    X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=valid_size, shuffle=False)

    # Étape 6 : Préparation des données pour le modèle RNN
    X_train = X_train.reshape(X_train.shape[0], sequence_length, len(features))
    X_valid = X_valid.reshape(X_valid.shape[0], sequence_length, len(features))
    X_test = X_test.reshape(X_test.shape[0], sequence_length, len(features))

    return X_train, y_train, X_valid, y_valid, X_test, y_test

### ENTRAINEMENT SUR LES DONNNEES DECADAIRES

#### Extreme

In [ ]:
#data_1dek=load_data_dek(str(Config.DATASET_DIR) + '/' + str(Config.DATA_1DEK))
data_1dek=load_data_dek(os.path.join(Config.DATASET_DIR,Config.DATA_1DEK))

In [ ]:
data_1dek.head()

In [ ]:
data_1dek['Label Secheresse'].unique()

#### RandonForrest

In [ ]:
grouped_data = data_1dek.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_model_ranf(group_data, '1decade', 'Sev_Extr', station)

#### BOOST

In [ ]:
grouped_data = data_ext_1dek.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_boost(group_data, '1decade', 'extr', station)

In [ ]:
resultat_compile

#### RNN

In [ ]:
data_ext_1dek.head()

In [ ]:
grouped_data = data_ext_1dek.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    print(f"Processing station: {station}")
    X_train, y_train, X_valid, y_valid, X_test, y_test = prepare_data_for_rnn(group_data, feature_columns=feature_rnn_dek, time_columns=["Year", "Month", "Decade"], target_column="Label Secheresse", sequence_length=5)
    create_rnn_model(X_train, y_train, X_valid, y_valid, X_test, y_test, '1decade', 'extr', station)
    print(f"Processing complete for station: {station}")

#### Severe

In [ ]:
data_sev_1dek=load_data_dek(str(Config.DATASET_DIR) + '/' + str(Config.DATA_1DEK_SEV))

#### RandonForrest

In [ ]:
grouped_data = data_sev_1dek.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_model_ranf(group_data, '1decade', 'sev', station)


#### BOOST

In [ ]:
grouped_data = data_sev_1dek.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_boost(group_data, '1decade', 'sev', station)


#### RNN

In [ ]:
grouped_data = data_sev_1dek.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    X_train, y_train, X_valid, y_valid, X_test, y_test = prepare_data_for_rnn(data_sev_1dek, feature_columns=feature_rnn_dek,time_columns=["Year", "Month", "Decade"], target_column="Label Secheresse", sequence_length=5)
    create_rnn_model(X_train, y_train,X_valid, y_valid,X_test, y_test,'1decade','sev',station)

### ENTRAINEMENT SPEI 1-MOIS

#### Extreme

In [ ]:
data_ext_1mois=load_data_mon(str(Config.DATASET_DIR) + '/' + str(Config.DATA_1MON_EXTR))

#### RandonForrest

In [ ]:
grouped_data = data_ext_1mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_model_ranf(group_data, '1mois', 'extr', station)

#### BOOST

In [ ]:
grouped_data = data_ext_1mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_boost(group_data, '1mois', 'extr', station)


#### RNN

In [ ]:
grouped_data = data_ext_1mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    X_train, y_train, X_valid, y_valid, X_test, y_test = prepare_data_for_rnn(data_ext_1mois, feature_columns=feature_rnn_mon,time_columns=["Year", "Month"], target_column="Label Secheresse", sequence_length=5)
    create_rnn_model(X_train, y_train,X_valid, y_valid,X_test, y_test,'1mois','extr',station)

#### Severe

In [ ]:
data_sev_1mois=load_data_mon(str(Config.DATASET_DIR) + '/' + str(Config.DATA_1MON_SEV))

#### RandonForrest

In [ ]:
grouped_data = data_sev_1mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_model_ranf(group_data, '1mois', 'sev', station)

#### BOOST

In [ ]:
grouped_data = data_sev_1mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_boost(group_data, '1mois', 'sev', station)


#### RNN

In [ ]:
grouped_data = data_sev_1mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    X_train, y_train, X_valid, y_valid, X_test, y_test = prepare_data_for_rnn(data_sev_1mois, feature_columns=feature_rnn_mon,time_columns=["Year", "Month"], target_column="Label Secheresse", sequence_length=5)
    create_rnn_model(X_train, y_train,X_valid, y_valid,X_test, y_test,'1mois','sev',station)

### ENTRAINEMENT SPEI 3-MOIS

#### Extreme

In [ ]:

data_ext_3mois=load_data_mon(str(Config.DATASET_DIR) + '/' + str(Config.DATA_3MON_EXTR))

#### RandonForrest

In [ ]:
grouped_data = data_ext_3mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_model_ranf(group_data, '3mois', 'extr', station)


#### BOOST

In [ ]:
grouped_data = data_ext_3mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_boost(group_data, '3mois', 'extr', station)

#### RNN

In [ ]:
grouped_data = data_ext_3mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    X_train, y_train, X_valid, y_valid, X_test, y_test = prepare_data_for_rnn(data_ext_3mois, feature_columns=feature_rnn_mon,time_columns=["Year", "Month"], target_column="Label Secheresse", sequence_length=5)
    create_rnn_model(X_train, y_train,X_valid, y_valid,X_test, y_test,'3mois','extr',station)

#### Severe

In [ ]:
data_sev_3mois=load_data_mon(str(Config.DATASET_DIR) + '/' + str(Config.DATA_3MON_SEV))

#### RandonForrest

In [ ]:
grouped_data = data_sev_3mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_model_ranf(group_data, '3mois', 'sev', station)

#### BOOST

In [ ]:
grouped_data = data_sev_3mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_boost(group_data, '3mois', 'sev', station)

#### RNN

In [ ]:
grouped_data = data_sev_3mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    X_train, y_train, X_valid, y_valid, X_test, y_test = prepare_data_for_rnn(data_sev_3mois, feature_columns=feature_rnn_mon,time_columns=["Year", "Month"], target_column="Label Secheresse", sequence_length=5)
    create_rnn_model(X_train, y_train,X_valid, y_valid,X_test, y_test,'3mois','sev',station)

### ENTRAINEMENT SPEI 6-MOIS

#### Extreme

In [ ]:

data_ext_6mois=load_data_mon(str(Config.DATASET_DIR) + '/' + str(Config.DATA_6MON_EXTR))

In [ ]:
data_ext_6mois.head()

#### RandonForrest

In [ ]:
grouped_data = data_ext_6mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_model_ranf(group_data, '6mois', 'extr', station)


#### BOOST

In [ ]:
grouped_data = data_ext_6mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_boost(group_data, '6mois', 'extr', station)

#### RNN

In [ ]:
data_ext_6mois.head()

In [ ]:
grouped_data = data_ext_6mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    X_train, y_train, X_valid, y_valid, X_test, y_test = prepare_data_for_rnn(data_ext_6mois, feature_columns=feature_rnn_mon,time_columns=["Year", "Month"], target_column="Label Secheresse", sequence_length=5)
    create_rnn_model(X_train, y_train,X_valid, y_valid,X_test, y_test,'6mois','extr',station)

#### Severe

In [ ]:
data_sev_6mois=load_data_mon(str(Config.DATASET_DIR) + '/' + str(Config.DATA_6MON_SEV))

#### RandonForrest

In [ ]:
#rand_sev_6mois=train_and_evaluate_model_ranf(data_sev_6mois)
grouped_data = data_sev_6mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_model_ranf(group_data, '6mois', 'sev', station)

#### BOOST

In [ ]:
grouped_data = data_sev_6mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_boost(group_data, '6mois', 'sev', station)

#### RNN

In [ ]:
grouped_data = data_sev_6mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    X_train, y_train, X_valid, y_valid, X_test, y_test = prepare_data_for_rnn(data_sev_6mois, feature_columns=feature_rnn_mon,time_columns=["Year", "Month"], target_column="Label Secheresse", sequence_length=5)
    create_rnn_model(X_train, y_train,X_valid, y_valid,X_test, y_test,'6mois','sev',station)

### ENTRAINEMENT SPEI 1-MOIS

In [ ]:
resultat_compile

In [ ]:
result=pd.DataFrame(resultat_compile)

In [ ]:
result.to_csv('./resultat.csv')
result.to_excel('./resultat.xlsx')